In [ ]:
# %pip install seaborn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mailbox
import re
import pytz
import seaborn as sns

In [9]:
!unzip /content/takeout-20230928T170442Z-001.zip

unzip:  cannot find either /content/takeout-20230928T170442Z-001.zip or /content/takeout-20230928T170442Z-001.zip.zip.


In [ ]:
mbox = mailbox.mbox("/content/Takeout/E-mail/Todos os e-mails, incluindo Spam e Lixeira.mbox")

mbox[0].keys()

In [ ]:
email_dict = {
    'date': [],
    'from': [],
    'to': [],
    'subject': [],
    'labels': []
}

# coletando os dados
for message in mbox:
    email_dict['date'].append(message['date'])
    email_dict['from'].append(message['from'])
    email_dict['to'].append(message['to'])
    email_dict['subject'].append(message['subject'])
    email_dict['labels'].append(message['X-Gmail-Labels'])

In [ ]:
df = pd.DataFrame(email_dict)
df.head()

In [ ]:
df.dtypes

In [ ]:
df['date'] = pd.to_datetime(df['date'], utc=True)

In [ ]:
df["date"] = df["date"].apply(lambda x: x.astimezone(pytz.timezone("America/Sao_Paulo")))
df.head()

In [ ]:
df.isnull().sum()
df.dropna(how='any', inplace=True)
df.isnull().sum()

In [ ]:
# criando a função
def extrair_email(string_email):
    email = re.findall(r'<(.+?)>', string_email)
    if email:
        return email[0]
    else:
        return string_email

In [ ]:
df['to'] = df['to'].apply(extrair_email)
df['from'] = df['from'].apply(extrair_email)
df.head()

In [ ]:
def extrair_label(string_labels):
    return string_labels.replace("=?UTF-8?Q?", "").split(",")[0]

df["labels"] = df["labels"].apply(extrair_label)

In [ ]:
df[df["labels"] == "Caixa_de_entrada"]["from"].value_counts()[0:5]

In [ ]:
df["weekday"] = df["date"].dt.weekday
dias_semana = {0:"Segunda", 1:"Terça", 2:"Quarta",3:"Quinta", 4:"Sexta", 5:"Sábado", 6:"Domingo"}
df["weekday"] = df["weekday"].replace(dias_semana)
df.head()

In [ ]:
sns.countplot(df["weekday"], order=["Segunda", "Terça", "Quarta", "Quinta", "Sexta", "Sábado", "Domingo"]);

In [ ]:
df["hora"] = df["date"].dt.hour
df.head()

In [ ]:
sns.countplot(df["hora"]);

In [ ]:
df['date'] = df['date'].apply(lambda a: pd.to_datetime(a).date())

In [ ]:
df.to_excel("meu_gmail.xlsx", index=False)